In [ ]:
# imports

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
data = pd.read_csv('data.csv')
numeric_attributes = data.select_dtypes(include=['float64', 'int64'])

In [ ]:
# Class equilibrium

plt.figure(figsize=(6, 4))
sns.countplot(x='Diagnostic', data=data)
plt.title('Class Frequency in the Dataset')
plt.xlabel('Diagnosis')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Numerical Attributes

mad = lambda x: np.mean(np.abs(x - np.mean(x)))
mead = lambda x: np.median(np.abs(x - np.median(x)))

for column in numeric_attributes.columns:
    print("Statistics for column", column)
    print("Mean:", numeric_attributes[column].mean())
    print("Std Deviation:", numeric_attributes[column].std())
    print("Mean Abs Deviation:", mad(numeric_attributes[column]))
    print("Min Val:", numeric_attributes[column].min())
    print("Max Val:", numeric_attributes[column].max())
    print("Amplitude:", numeric_attributes[column].max() - numeric_attributes[column].min())
    print("Median:", numeric_attributes[column].median())
    print("Median Abs Deviation:", mead(numeric_attributes[column]))
    q1 = numeric_attributes[column].quantile(0.25)
    q3 = numeric_attributes[column].quantile(0.75)
    print("Interquantil Interval:", q3 - q1)
    print()

plt.figure(figsize=(8, 6))
numeric_attributes.hist(bins=20, figsize=(12, 6))
plt.suptitle('Numerical Attributes Histogram', fontsize=16)
plt.show()

In [ ]:
# Covariance Matrix

covariance_matrix = numeric_attributes.corr()
plt.figure(figsize=(6, 4))
sns.heatmap(covariance_matrix, annot=True, cmap='coolwarm')
plt.title('Covariance Matrix')
plt.show()

plt.figure(figsize=(10, 8))
sns.pairplot(data, hue='Diagnostic', diag_kind='kde')
plt.title('Pair Diagram and Class Distribution')
plt.show()